<a href="https://colab.research.google.com/github/jihun5/data-preprocessing_/blob/main/%EB%AA%A8%EB%8D%B8%EC%9E%91%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

bank_df = pd.read_csv('bank-prep.csv', sep=',')
bank_df.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,dec,feb,jan,jul,jun,mar,may,nov,oct,sep
0,58,0,2143,1,0,5,261,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
1,36,0,265,1,1,5,348,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
2,25,0,-7,1,0,5,365,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
3,53,0,-3,0,0,5,1666,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
4,24,0,-103,1,1,5,145,1,-1,0,...,0,0,0,0,0,0,1,0,0,0


In [2]:
# yes와 no의 개수 차이로 인하여 정확한 측정에 어려움이 있을 수 있다.
# 다운샘플링 진행
!pip show imbalanced-learn

Name: imbalanced-learn
Version: 0.10.1
Summary: Toolbox for imbalanced dataset in machine learning.
Home-page: https://github.com/scikit-learn-contrib/imbalanced-learn
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scikit-learn, scipy, threadpoolctl
Required-by: 


In [3]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.8.6
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.13.1
backcall                         0.2.0
beautifulsoup4                   4.11.2
bidict                           0.22.1
b

In [7]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
X = np.array(bank_df.drop('y', axis=1))
Y = np.array(bank_df[['y',]])
print(np.sum(Y == 1), np.sum(Y == 0))

sampler = RandomUnderSampler(random_state=42)
X, Y = sampler.fit_resample(X,Y)
print(np.sum( Y==1), np.sum(Y==0))

820 6113
820 820


In [9]:
# 오버 샘플링
from imblearn.over_sampling import RandomOverSampler
'''
X = np.array(bank_df.drop('y', axis=1))
Y = np.array(bank_df[['y']])
print(np.sum(Y==1), np.sum(Y==0))

sampler = RandomOverSampler(random_state=42)
X, Y = sampler.fit_resample(X,Y)
print(np.sum(Y==1), np.sum(Y==0))
'''

"\nX = np.array(bank_df.drop('y', axis=1))\nY = np.array(bank_df[['y']])\nprint(np.sum(Y==1), np.sum(Y==0))\n\nsampler = RandomOverSampler(random_state=42)\nX, Y = sampler.fit_resample(X,Y)\nprint(np.sum(Y==1), np.sum(Y==0))\n"

복사시키지 않고 보완해서 데이터를 증가시키는 방법 SMOTE

SMOTE는 K-NN법 아이디어를 사용한다. 데이터 점 사이의 이웃 근접도는 유클리드 거리를 사용하여 두 점 간의 직선거리를 계산한다.

SMOTE는 수가 적은 클래스에 주목해서 어떤 데이터 점과 근접한 데이터 점 간에 새로운 데이터 점 간에 새로운 데이터 점을 생성하는 것이다.

In [11]:
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=10, shuffle=True)
scores = []  # 점수를 저장할 리스트

for train_id, test_id in kf.split(X):
    x_train, x_test = X[train_id], X[test_id]
    y_train, y_test = Y[train_id], Y[test_id]

    clf = tree.DecisionTreeClassifier()
    clf.fit(x_train, y_train)
    pred_y = clf.predict(x_test)

    # accuracy_score 함수에 실제 레이블과 예측 레이블을 모두 전달
    acc_score = accuracy_score(y_test, pred_y)
    scores.append(acc_score)  # 점수를 리스트에 추가

scores = np.array(scores)
print(scores.mean(), scores.std())

0.7737804878048781 0.040671234501590964


In [13]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print(recall_score(Y[test_id], pred_y))
print(precision_score(Y[test_id], pred_y))

0.8444444444444444
0.8172043010752689


In [14]:
print(clf)

DecisionTreeClassifier()


In [15]:
from sklearn.model_selection import GridSearchCV

params = {
    'criterion' : ['entropy', 'gini'],
    'max_depth' : [2,4,6,8,10],
    'min_samples_leaf' : [10,20,30,40,50],
}

clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params, cv=KFold(n_splits=10, shuffle=True), scoring='accuracy')
clf_gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [10, 20, 30, 40, 50]},
             scoring='accuracy')

In [16]:
print(clf_gs.best_score_)
print(clf_gs.best_params_)

0.8036585365853659
{'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 10}


In [17]:
clf_best = tree.DecisionTreeClassifier(criterion = 'gini', max_depth=8, min_samples_leaf=10)
clf_best.fit(X,Y)
print(clf_best.feature_importances_)

[3.07542827e-02 0.00000000e+00 4.52910472e-02 6.09719622e-02
 0.00000000e+00 2.63786873e-02 4.79842806e-01 2.40464200e-02
 1.83293639e-01 0.00000000e+00 0.00000000e+00 1.97259206e-02
 1.67089056e-03 2.01733315e-04 1.32957772e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.26420316e-02 7.71319856e-03
 0.00000000e+00 0.00000000e+00 1.53273002e-03 0.00000000e+00
 7.46401294e-04 0.00000000e+00 4.90339049e-02 1.28644245e-02
 1.96034263e-03 0.00000000e+00 0.00000000e+00]
